In [1]:
import sys
sys.path = ['../../src'] + sys.path # this ensures the notebook uses the source version of Tarski

In [2]:
import tarski
from tarski.theories import Theory
from tarski.syntax import *
from tarski.io import rddl
from tarski.model import Model
from tarski.evaluators.simple import evaluate
from tarski.syntax.arithmetic import *
from tarski.syntax.arithmetic.special import *
from tarski.syntax.arithmetic.random import *

In [3]:
import numpy

# Modeling a reach task with a Double Integrator in Tarski

In this tutorial we will show how we can specify a simple linear dynamic system (or _plant_) using [scaling chains of integrators](https://fmrchallenge.org/integrator_chains.html). In particular, we will consider a fully observable, noise-free _double integrator_. 

Such specifications are one of the most fundamental systems in control applications, representing single-degree-of-freedom translational and rotational motion. Applications of the double integrator include low-friction, free rigid-body motion, such as single-axis spacecraft rotation and rotary crane motion. Control of the double integrator has been of interest since the early days of control theory when it was used extensively to illustrate minimum-time and minimum-fuel controllers. For further background, check out [this reference](http://ieeecss.org/CSM/library/2001/oct01/naivecontrol.pdf). 

We start defining the first order language that we'll use to define state, control variables and the constraints

In [4]:
lang = tarski.language('double_integrator', [Theory.EQUALITY, Theory.ARITHMETIC, Theory.SPECIAL, Theory.RANDOM])

Note that we're loading theories for equality, arithmetics, special functions and random variates. 

Since we're modeling the behaviour of one or more vehicles, and possibly obstacles too, we define the sort _vehicle_ to organise our objects

In [5]:
vehicle = lang.sort('vehicle', lang.Object)

## States

We describe the position and velocities of vehicles with four functions that map vehicles to the reals

In [6]:
# state variables
x = lang.function('x', vehicle, lang.Real)
y = lang.function('y', vehicle, lang.Real)
vx = lang.function('vx', vehicle, lang.Real)
vy = lang.function('vy', vehicle, lang.Real)

Control inputs - the second derivatives or _accelerations_ - are represented also with two functions

In [7]:
# control variables
ux = lang.function('ux', vehicle, lang.Real)
uy = lang.function('uy', vehicle, lang.Real)

In this tutorial we will consider a single vehicle, which we declare like this

In [8]:
# object
v_1 = lang.constant('v_1', vehicle)

## Defining the Task

A very general class of tasks are those corresponding to so--called _Reach-Avoid_ specifications that can be represented with the following LTL formula

$$
q(0) = x_0 \land (\square q(t) \notin Obs) \land \bigwedge_i \square \lozenge goal_i
$$

where $q(t)$ is the _configuration_ of the system (e.g. valuations of the terms describing the system state and inputs) at time $t$, $x_0$ is an arbitrary initial configuration, $Obs$ is the _obstacle set_ respresented by the finite union of the volume of geometric objects representing ``no-go`` areas, and $goal_i$ is a formula over the terms making up $q(t)$ that is verified by any configuration _within_ the volume of a geometric object centered at the _goal_ or _target_ region.

More generally, the second and third parts of the specification above are referred to as _state invariants_ or _state constraints_, and _goal constraint_, respectively.

In this example we do not have obstacles, so we only need to define the initial configuration and the goal constraint. For the latter, we need to introduce two new functions

In [9]:
# target
gx = lang.function('gx', lang.Real)
gy = lang.function('gy', lang.Real)

that designate the coordinates of the goal. We define their value and that of the initial configuration $q(0)$ by means of constructing a _model_ 

In [10]:
x0 = Model(lang)
x0.evaluator = evaluate

and setting the values associated with each of the terms that make up the description of the system. To wit

- Initial position and velocity of the vehicle $v$, e.g. $x(v)$, $y(v)$, $vx(v)$ and $vy(v)$

In [11]:
x0.set(x(v_1), 0.5) # x(v_1) = 0.5
x0.set(y(v_1), 0.5) # y(v_1) = 0.5
x0.set(vx(v_1), 0.0) # vx(v_1) = 0.5
x0.set(vy(v_1), 0.0) # vy(v_1) = 0.5

 - Initial control inputs

In [12]:
x0.set(ux(v_1), 0.0) # ux(v_1) = 0.0
x0.set(uy(v_1), 0.0) # uy(v_1) = 0.0

- And the coordinates of the goal

In [13]:
x0.set(gx(), 5.5)
x0.set(gx(), 2.0)

For the goal constraint, we choose to represent the states deemed as acceptable terminal states by specifying an axis aligned box centered on $(g_x, g_y)$

$$goal_0 \equiv (g_x - 0.5) \leq x(v_{1}) \leq (g_x + 0.5)$$

In [14]:
goal_0 = ((gx() - 0.5) <= x(v_1)) & (x(v_1) <= (gx() + 0.5))

$$goal_1 \equiv (g_y - 0.5) \leq y(v_{1}) \leq (g_y + 0.5)$$

In [15]:
goal_1 = ((gy() - 0.5) <= y(v_1)) & (y(v_1) <= (gy() + 0.5))

Verifying that the ``eventually`` ($\lozenge$) in the goal specification is satisfied by a configuration $q(t)$ amounts to evaluating expression

$$[\bigwedge_i goal_i]^{q(t)}$$

where each of the terms in $goal_1$ and $goal_0$ are replaced by the values assigned by the configuration $q(t)$. We can do this for $q(0)$ which is implicitly represented by the model object ```x0```

In [16]:
x0[goal_0 & goal_1]

False

which turns out to be false.

Ensuring that the $\square$ in goal constraint is more complicated. The property represented by the temporal operator is that our system remains in any state _within_ the region around the goal _infinitely often_. This may prove to be impossible when there is a random _perturbation_ making a contribution to the evolution of the state variables over time. 

Also for some systems, by discretizing time $t$, it may be the case that the duration of the control cycle and discretization time step, is such that it is not possible to steer the system in a timely fashion so as to keep it inside the region around the goal.

In this tutorial we relax slightly the requirements posed  by $\square$, so that the configurations $q(t)$ the system is in are _as often as possible_ within the target region. For that, we will define a quadratic cost function

$$ J = \sum_{k} (q(k)-G)^T Q (q(k)-G)$$

where Q is a positive definite matrix, and G is the column vector $[g_x\ g_y]^T$. Optimal inifite sequences of control inputs $U^{*}$ that _maximize_ the number of time steps that the system is inside the target region are those that minimize $J$. We define the cost matrix $Q$ as follows

In [17]:
Q = lang.matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], lang.Real)

Now we need to arrange the terms describing configurations $q()$ and the goal as vectors

In [18]:
q = lang.vector([x(v_1), y(v_1), vx(v_1), vy(v_1)], lang.Real)

and

In [19]:
G = lang.vector([gx(), gy(), 0.0, 0.0], lang.Real)

The instantaneous cost can then be written as follows

In [20]:
stage_cost = transpose(q-G) * Q * (q-G)

if we print ```stage_cost``` we will get the expression represented as a series of nested arithmetic operations in in prefix notation

In [21]:
print(stage_cost)

*(*(-([[x(v_1) y(v_1) vx(v_1) vy(v_1)]], [[gx() gy() 0.0 (Real) 0.0 (Real)]]), [[1.0 (Real) 0.0 (Real) 0.0 (Real) 0.0 (Real)]
 [0.0 (Real) 1.0 (Real) 0.0 (Real) 0.0 (Real)]
 [0.0 (Real) 0.0 (Real) 0.0 (Real) 0.0 (Real)]
 [0.0 (Real) 0.0 (Real) 0.0 (Real) 0.0 (Real)]]), -([[x(v_1)]
 [y(v_1)]
 [vx(v_1)]
 [vy(v_1)]], [[gx()]
 [gy()]
 [0.0 (Real)]
 [0.0 (Real)]]))


We note that the above is not meant to be readable, ```Tarski``` aims at offering an abstract, flexible representation for constraints and expressions to interface with third-party solvers, as well as some tools to ease the task of _compiling_ ```Tarski``` syntactic trees into the languages/formalisms supported by planners, SMT solvers or mathematical programming suites.

## Constraints

The specification above is of little use without a definition of an underlying transition system. For that, we will discretize the set of (linear) ordinary differential equations 

$$
\dot{q} = A q + B u
$$

with the explicit Euler method, so that

$$
q(t) = q(t-1) +  (A q(t-1) + B u)\Delta t
$$

where $\Delta t$, or discretization step, is the integration time step as well.

### Dynamics

We start defining $\Delta t$

In [22]:
dt = lang.function('dt', lang.Real)

The _state matrix_ $A$ describes how the state variables interact with each other, modeling this interaction as a _linear combination_. 

In [23]:
A = lang.matrix([[0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]], lang.Real)

In [24]:
print(A)

[[0.0 (Real) 0.0 (Real) 1.0 (Real) 0.0 (Real)]
 [0.0 (Real) 0.0 (Real) 0.0 (Real) 1.0 (Real)]
 [0.0 (Real) 0.0 (Real) 0.0 (Real) 0.0 (Real)]
 [0.0 (Real) 0.0 (Real) 0.0 (Real) 0.0 (Real)]]


the matrix above indicates that the _couplings_ between state variables are a very simple graph made of two connected components (two chains, with two vertices each). We can obtain the equations for the _autonomous response_ of the system multiplying the ```numpy``` matrices we use to implement these groups of terms

In [25]:
homogenous=simplify(A.matrix * q.matrix)

for i in range(q.shape[0]):
    print('delta {} / delta t = {}'.format(q.matrix[i,0], homogenous[i,0]))

delta x(v_1) / delta t = vx(v_1)
delta y(v_1) / delta t = vy(v_1)
delta vx(v_1) / delta t = 0.0
delta vy(v_1) / delta t = 0.0


note the use of the function ```simplify()``` to obtain a "clean" expression for the derivatives.

The response to the inputs is given by the following matrix

In [26]:
B = lang.matrix([[0, 0], [0, 0], [1, 0], [0, 1]], lang.Real)

that indicates that inputs only affect the velocities

In [27]:
print(B)

[[0.0 (Real) 0.0 (Real)]
 [0.0 (Real) 0.0 (Real)]
 [1.0 (Real) 0.0 (Real)]
 [0.0 (Real) 1.0 (Real)]]


We group the terms for the system inputs in a column vector

In [28]:
u = lang.vector([ux(v_1), uy(v_1)], lang.Real)

and obtain the full set of differential equations

In [29]:
non_homogenous=simplify(B.matrix * u.matrix)

for i in range(q.shape[0]):
    print('delta {} / delta t = {}'.format(q.matrix[i,0], simplify(homogenous[i,0] + non_homogenous[i,0])))

delta x(v_1) / delta t = vx(v_1)
delta y(v_1) / delta t = vy(v_1)
delta vx(v_1) / delta t = ux(v_1)
delta vy(v_1) / delta t = uy(v_1)


Alternatively, the dynamic constraint can be expressed as well in its abstract form, enabling alternative rewritings

In [31]:
A * q + dt() * (B * u)

+(*([[0.0 (Real) 0.0 (Real) 1.0 (Real) 0.0 (Real)]
 [0.0 (Real) 0.0 (Real) 0.0 (Real) 1.0 (Real)]
 [0.0 (Real) 0.0 (Real) 0.0 (Real) 0.0 (Real)]
 [0.0 (Real) 0.0 (Real) 0.0 (Real) 0.0 (Real)]], [[x(v_1)]
 [y(v_1)]
 [vx(v_1)]
 [vy(v_1)]]), *(dt(), *([[0.0 (Real) 0.0 (Real)]
 [0.0 (Real) 0.0 (Real)]
 [1.0 (Real) 0.0 (Real)]
 [0.0 (Real) 1.0 (Real)]], [[ux(v_1)]
 [uy(v_1)]])))